In [25]:
from os import listdir # to get the files and directories
from os.path import join, isdir # to joint base path with sub path
import numpy as np # numpy library will help us to work with the numbers and arrays
import matplotlib.pyplot as plt # to visualize/plot the images
import cv2 # handle the images
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Input,
    Dense,
    Conv2D,
    MaxPooling2D,
    Flatten,
    Dropout,
)
from scikeras.wrappers import KerasClassifier
from time import time

In [4]:
base_path = "/kaggle/input/the-iqothnccd-lung-cancer-dataset/The IQ-OTHNCCD lung cancer dataset"
categories = listdir(base_path)
categories = [i for i in categories if isdir(join(base_path, i))]

In [5]:
images = {}
for category in categories:
    images[category] = []
    category_path = join(base_path, category)
    for img_name in listdir(category_path):
        img_path = join(category_path, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        images[category].append(img)

In [6]:
def resize_images(target_size):
    for category in categories:
        for index, img in enumerate(images[category]):
            if img.shape != target_size:
                images[category][index] = cv2.resize(img, target_size)


resize_images((128, 128))

In [7]:
for category in categories:
    images[category] = np.array(images[category])

In [8]:
X = np.concatenate(list(images.values()))
y = np.concatenate([[category]*len(images[category]) for category in categories])

In [9]:
one_class = ["Malignant cases", "Non-Malignant cases"]
y_one_class = np.where(y == one_class[0], one_class[0], one_class[1])

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y_one_class, test_size=0.3, stratify=y_one_class, random_state=42)

In [11]:
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

In [12]:
X_train = X_train.reshape(-1,*X_train[0].shape, 1)

In [13]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [14]:
def create_model():
    model = Sequential()
    # Input Layer
    model.add(Input(shape=X_train[0].shape))
    # Hidden Layers
    model.add(Conv2D(filters=64, kernel_size=(3, 3), name="conv2d_1", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), name="max2d_1"))

    model.add(Conv2D(filters=32, kernel_size=(3, 3), name="conv2d_2", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), name="max2d_2"))

    model.add(Conv2D(filters=8, kernel_size=(3, 3), name="conv2d_3", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), name="max2d_3"))

    model.add(Flatten(name="flatten"))

    # Output Layer
    model.add(Dense(1, activation="sigmoid", name="out"))
    return model

## Grid Search

In [24]:
# define the grid search parameters
batch_size = [32, 64, 128]
epochs = [15, 20, 30]
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3, 0.05]
optimizer = ["RMSprop", "adam"]
param_grid = dict(
    batch_size=batch_size,
    epochs=epochs,
    optimizer__learning_rate=learn_rate,
    optimizer=optimizer,
)
grid = GridSearchCV(estimator=model, param_grid=param_grid)

In [ ]:
start = time()
grid_result = grid.fit(X_train, y_train_encoded)
end = time()

Epoch 1/15
13/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5726 - loss: 0.6742

2024-06-16 22:25:22.914186: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 4.1327, expected 3.479
2024-06-16 22:25:22.914243: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1: 4.44401, expected 3.79031
2024-06-16 22:25:22.914253: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 2: 4.70176, expected 4.04806
2024-06-16 22:25:22.914261: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 4.03011, expected 3.37641
2024-06-16 22:25:22.914270: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4: 3.86409, expected 3.21039
2024-06-16 22:25:22.914278: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 5: 4.46, expected 3.8063
2024-06-16 22:25:22.914286: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 6: 3.67012, expected 3.01642
2024-06-16 22:25:22.914295: E external/local_xla/xla/service/g

20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 86ms/step - accuracy: 0.5871 - loss: 0.6676
Epoch 2/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6803 - loss: 0.5885
Epoch 3/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7446 - loss: 0.5307
Epoch 4/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7513 - loss: 0.4625
Epoch 5/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8208 - loss: 0.4025
Epoch 6/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8416 - loss: 0.3583
Epoch 7/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8836 - loss: 0.2926
Epoch 8/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8941 - loss: 0.2617
Epoch 9/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8914 - loss: 0.2451
Epoch 10/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9199 - loss: 0.2285
Epoch 11/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9230 - loss: 0.2464
Epoch 12/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9403 - loss: 0.1687

2024-06-16 22:25:27.399756: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 3.13985, expected 2.67636
2024-06-16 22:25:27.399804: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1: 3.35193, expected 2.88844
2024-06-16 22:25:27.399813: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 2: 2.99834, expected 2.53485
2024-06-16 22:25:27.399821: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 3.17316, expected 2.70967
2024-06-16 22:25:27.399829: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4: 3.37558, expected 2.91209
2024-06-16 22:25:27.399837: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 5: 2.76669, expected 2.3032
2024-06-16 22:25:27.399845: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 6: 2.12022, expected 1.65674
2024-06-16 22:25:27.399852: E external/local_xla/xla/ser

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 283ms/step
Epoch 1/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - accuracy: 0.5622 - loss: 0.6769
Epoch 2/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7134 - loss: 0.6112
Epoch 3/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7589 - loss: 0.5273
Epoch 4/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7465 - loss: 0.4814
Epoch 5/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7776 - loss: 0.4355
Epoch 6/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8427 - loss: 0.3796
Epoch 7/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8719 - loss: 0.3218
Epoch 8/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8734 - loss: 0.3004
Epoch 9/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8891 - loss: 0.2801
Epoch 10/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8996 - loss: 0.2599
Epoch 11/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9028 - loss: 0.2257
Epoch 12/15
20/20 ━━━━━━━━━━━━━━━━

...

3/5 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.5195 - loss: 0.6906

2024-06-16 23:08:09.189013: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 3.72387, expected 3.04068
2024-06-16 23:08:09.189068: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1: 3.627, expected 2.94381
2024-06-16 23:08:09.189077: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 2: 4.53293, expected 3.84974
2024-06-16 23:08:09.189085: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 3.19391, expected 2.51072
2024-06-16 23:08:09.189092: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4: 3.1144, expected 2.43121
2024-06-16 23:08:09.189100: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 5: 4.08324, expected 3.40006
2024-06-16 23:08:09.189108: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 6: 2.98628, expected 2.30309
2024-06-16 23:08:09.189115: E external/local_xla/xla/servi

5/5 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.5194 - loss: 0.6872
Epoch 2/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.6665 - loss: 0.6492
Epoch 3/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.6506 - loss: 0.6087
Epoch 4/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.6981 - loss: 0.5619
Epoch 5/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.7160 - loss: 0.5447
Epoch 6/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.7310 - loss: 0.5146
Epoch 7/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.7466 - loss: 0.5152
Epoch 8/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.7461 - loss: 0.4964
Epoch 9/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.7976 - loss: 0.4313
Epoch 10/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.7878 - loss: 0.4422
Epoch 11/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.8126 - loss: 0.4363
Epoch 12/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.8331 - loss: 0.3880
Epoch 13/15
5

2024-06-16 23:08:24.655366: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 15876: 2.40802, expected 1.6456
2024-06-16 23:08:24.655422: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 15877: 2.25118, expected 1.48876
2024-06-16 23:08:24.655440: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 15878: 2.41285, expected 1.65043
2024-06-16 23:08:24.655458: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 15879: 2.4824, expected 1.71998
2024-06-16 23:08:24.655471: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 15880: 2.29272, expected 1.5303
2024-06-16 23:08:24.655495: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 15881: 1.84768, expected 1.08526
2024-06-16 23:08:24.655507: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 15882: 2.1451, expected 1.38268
2024-06-16 23:08:24.655517: E e

5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.5415 - loss: 0.6838 
Epoch 2/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5937 - loss: 0.6457
Epoch 3/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.6486 - loss: 0.6051
Epoch 4/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.7094 - loss: 0.5623
Epoch 5/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.7121 - loss: 0.5426
Epoch 6/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.7532 - loss: 0.5071
Epoch 7/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.7301 - loss: 0.5250
Epoch 8/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.7421 - loss: 0.5032
Epoch 9/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.7776 - loss: 0.4527
Epoch 10/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.7754 - loss: 0.4492
Epoch 11/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.8063 - loss: 0.4231
Epoch 12/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.8075 - loss: 0.4222
Epoch 13/15
5

In [27]:
print(grid_result.best_params_)

{'batch_size': 32, 'epochs': 30, 'optimizer': 'adam', 'optimizer__learning_rate': 0.001}


In [36]:
print(grid_result.best_score_)

0.9765384941855529


In [35]:
t = end-start
h = t//3600
m = t//60 - h*60
print(f"Running Time: {h:0.0f} hour {m:0.0f} minutes")

Running Time: 1 hour 3 minutes


In [39]:
result_df = pd.DataFrame(grid_result.cv_results_)
result_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_batch_size,param_epochs,param_optimizer,param_optimizer__learning_rate,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,5.473498,0.426241,0.892614,0.452693,32,15,RMSprop,0.001,"{'batch_size': 32, 'epochs': 15, 'optimizer': ...",0.805195,0.928571,0.967320,0.934641,0.888889,0.904923,0.055747,11
1,5.268247,0.235219,0.512809,0.007702,32,15,RMSprop,0.010,"{'batch_size': 32, 'epochs': 15, 'optimizer': ...",0.681818,0.675325,0.509804,0.549020,0.640523,0.611298,0.069435,61
2,5.078763,0.062141,0.517525,0.008325,32,15,RMSprop,0.100,"{'batch_size': 32, 'epochs': 15, 'optimizer': ...",0.714286,0.487013,0.718954,0.712418,0.647059,0.655946,0.088521,39
3,5.157933,0.195876,0.519118,0.007999,32,15,RMSprop,0.200,"{'batch_size': 32, 'epochs': 15, 'optimizer': ...",0.629870,0.655844,0.790850,0.509804,0.535948,0.624463,0.099680,49
4,5.126770,0.125057,0.511028,0.004206,32,15,RMSprop,0.300,"{'batch_size': 32, 'epochs': 15, 'optimizer': ...",0.577922,0.688312,0.509804,0.751634,0.529412,0.611417,0.093537,60


In [40]:
result_df.to_csv("grid_search_result.csv")

In [44]:
# Final Model
model = grid_result.best_estimator_.model

In [45]:
model.evaluate(X_test, y_test_encoded)

11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.9892 - loss: 0.0225


[0.015378828160464764, 0.9909090995788574]

In [48]:
# Saving the model
model.save("lung_cancer_final_model.h5")

In [49]:
le.classes_

array(['Malignant cases', 'Non-Malignant cases'], dtype='<U19')

In [ ]:
# Predicting
# 1. resize the image to (128,128) using bicubic interpolation
# 2. reshape the image to (-1, 128,128, 1)
# 3. rescale the value /255.0
# 4. get the prediction (probability of the positive class)
# 5. round the probability 
# 6. use le.inverse_transform to get the predicted class
# 7. return the predicited class with the probability
